In [ ]:
import sys
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import sklearn.linear_model
import geopy.distance

import utils

In [ ]:
df = pd.read_feather('SRFG-v1.fth')

# Constant predictions

In [ ]:
p = pd.DataFrame(index=df['datarate'].index)
p['const(mean)'] = np.repeat(df['datarate'].mean(), len(df))
p['const(median)'] = np.repeat(df['datarate'].median(), len(df))

# Basic lookups

In [ ]:
for v in ["sinr", "signal", "rsrq", "rsrp", "rssi"]:
    lookup = df.groupby(v, dropna=False)['datarate'].median()
    p[v+'-Lookup'] = list(lookup[df[v]])
    print(v, df[v].isna().sum())
    lookup.plot()
    plt.show()

In [ ]:
for i in range(2, 6):
    print(geopy.distance.geodesic((df["lat"].mean(), df["long"].mean()), (df["lat"].mean(), df["long"].mean()+10**-i)).meters)
    lookup = df.groupby(round(df.long, i), dropna=False)['datarate'].median()
    p['long-Lookup'+str(i)] = list(lookup[round(df["long"], i)])
    lookup.plot()
    plt.show()

# Estimation from other data rates

In [ ]:
p['prevDR'] = (df["datarate"].shift(1) * df["predecessor"]).fillna(df["datarate"].shift(1) * df["predecessor2"]).fillna(df["datarate"].mean())

In [ ]:
tp = (df["datarate"].shift(1) * df["predecessor"]).fillna(df["datarate"].shift(1) * df["predecessor2"])
ts = (df["datarate"].shift(-1) * df["predecessor"].shift(-1)).fillna(df["datarate"].shift(-1) * df["predecessor2"].shift(-1))
p['meanDR'] = ((tp+ts)/2).fillna(tp).fillna(ts).fillna(df["datarate"].mean())

# Evaluation

In [ ]:
r = utils.evaluate(p, df['datarate'])
r["Strategy"] = "\texttt{" + r["Strategy"] + "}"
r["MAE [\\si{\\mega\\bit\\per\\second}]"] = r["MAE"]/1e6
r["$R^2$"] = r["R2"]
print(r[["Strategy", "MAE [\\si{\\mega\\bit\\per\\second}]", "$R^2$"]].to_latex(index=False, float_format="%.2f", escape=False))